# Model

## Import and read Data

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import os
from train_tabular import fit

os.environ["WANDB_SILENT"] = "true"

X_train = pd.read_csv(r'../../data/train_data.csv')
Y_train = X_train["Category"]

X_test = pd.read_csv(r'../../data/test_data.csv')
Y_test = X_test["Category"]
X_train= X_train.iloc[:,[i for i in range(6,15)]+ [-1]]
X_test= X_test.iloc[:,[i for i in range(6,15)]]

## Preprocessing

In [ ]:
# define preprocessing function
X_train.loc[X_train['Patient_gender'] =='male','Patient_gender'] =1
X_train.loc[X_train['Patient_gender'] =='female','Patient_gender'] =0
X_test.loc[X_test['Patient_gender'] =='male','Patient_gender'] =1
X_test.loc[X_test['Patient_gender'] =='female','Patient_gender'] =0

## Setup Model

In [ ]:
model = RandomForestClassifier(n_estimators=100,random_state=42)

In [ ]:
wandbadd={'n_estimators':100,'random_state':42}

## Fit Model and Evaluate Model

In [ ]:
models = []
for fold in range(0,5):
    fitted_model_fold = fit(model,X_train,Y_train,X_test,Y_test,fold,"Tab-Data-RandomForest-0.9Train","RandomForest",None,wandb_additional_config=wandbadd)
    models.append(fitted_model_fold)

In [ ]:
fitted_model_fold = fit(model,X_train,Y_train,X_test,Y_test,'all',"Tab-Data-RandomForest-All-0.9Train","RandomForest",wandb_additional_config=wandbadd)